In [1]:
# !conda install -y -c conda-forge fbprophet
# !pip install mlflow
# !pip install --upgrade pandas
import sagemaker
import pandas as pd
import s3fs
from fbprophet import Prophet
import mlflow
from mlflow.pyfunc import log_model
import mlflow.pyfunc
from fbprophet.diagnostics import cross_validation, performance_metrics
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

Importing plotly failed. Interactive plots will not work.


In [2]:
bucket = sagemaker.Session().default_bucket()
# file_suffix = '_daily_raw_with_3_mo_target.csv'
folder = 'stock_data_raw_with_3_mo_target'
fs = s3fs.S3FileSystem()
files = fs.ls(f's3://{bucket}/{folder}')
try:
    exp_id = mlflow.create_experiment('fbprophet_all_')
except Exception as e:
    display(e)

In [3]:
class FbProphetWrapper(mlflow.pyfunc.PythonModel):

    def __init__(self, model):
        self.model = model
        super().__init__()


    def load_context(self, context):
        from fbprophet import Prophet
        return

    def predict(self, context, model_input):
        future = self.model.make_future_dataframe(periods=model_input['periods'][0])
        return self.model.predict(future)

In [ ]:
# for file in files[:1]:

metric = 'high' ##'low_window_20_shift_0_mean_percent_change_percent_lag_90_shift_-90'
rolling_window=0.1
# results = pd.DataFrame()
# results.columns = ['file', 'metric' ]
for file in files:
    with mlflow.start_run(experiment_id=exp_id):
        mlflow.log_param('file', file)
        mlflow.log_param('metric',metric)
    #     mlflow.log_param('rolling_window', rolling_window)
        dft = pd.read_csv(f's3://{file}')#.dropna()
        df = dft[['Unnamed: 0', metric]]
        df.columns = ['ds', 'y']
    #     display(df)
        m = Prophet(growth='linear')
        m.fit(df)
    #     log_model('model', FbProphetWrapper(m))
    #     df_cv = cross_validation(m, horizon = '90 days')
    #     df_p = performance_metrics(df_cv, rolling_window=rolling_window)
    #     mlflow.log_metric("rmse", df_p.loc[0,'RMSE'])
    #     mlflow.log_metric("MSE", df_p.loc[0,'MSE'])
    #     mlflow.log_metric("MAE", df_p.loc[0,'MAE'])
    #     mlflow.log_metric("MAPE", df_p.loc[0,'MAPE'])
    #     mlflow.log_metric("MDAPE", df_p.loc[0,'MDAPE'])
    #     mlflow.log_metric("SMAPE", df_p.loc[0,'SMAPE'])
    #     mlflow.log_metric("COVERAGE", df_p.loc[0,'COVERAGE'])   
        periods=90
        future = m.make_future_dataframe(periods=periods)
        mlflow.log_param('length of predeiction', periods)
    #     display(future)
        forecast = m.predict(future)
        yhat = forecast.loc[len(forecast.index)-1, 'yhat']
        y = df.loc[len(df.index)-1, 'y']
        percent_change = (yhat-y)/y
        mlflow.log_metric('percent change', percent_change)

    #     fig1 = m.plot(forecast)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/opt/conda/lib/python3.7/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/opt/conda/lib/python3.7/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [23]:
forecast.loc[len(forecast.index)-1, 'yhat']

2.155645709745646

In [6]:
runs = mlflow.search_runs(exp_id)


In [7]:
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.percent change,metrics.length of predeiction,params.file,params.metric,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.source.name
0,01d79f8370284999aa3c9f0bf29d55c3,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:06:41.293000+00:00,2021-04-21 03:06:47.425000+00:00,-0.550051,90.0,sagemaker-us-east-2-287085956542/stock_data_ra...,high,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
